In [29]:
import os
import numpy as np
import jieba
import chromadb
import requests
from rank_bm25 import BM25Okapi
from typing import List, Dict
from zai import ZhipuAiClient
import re

# 智谱Embedding3配置
class ZhipuEmbeddingFunction:
    """智谱Embedding3嵌入函数"""
    """（添加name属性）"""
    def name(self):
        """返回嵌入模型名称（Chroma要求的方法）"""
        return "zhipu-embedding-3"  # 关键修复：将name改为方法
    def __call__(self, input: List[str]) -> List[List[float]]:
        self.zhipu_api_key = os.getenv("ZHIPU_API_KEY")
        client = ZhipuAiClient(api_key= self.zhipu_api_key)
        response = client.embeddings.create(
        model="embedding-3", #填写需要调用的模型编码
        input=input,
        )
        return [data_point.embedding for data_point in response.data]
    def embed_query(self, input: str) -> List[float]:
        """为查询生成嵌入向量"""
        return self.__call__(input)


class updater:
    def __init__(self, chroma_persist_dir: str = "law_chroma_db"):
        self.workdir = os.getcwd()
        chroma_persist_path = os.path.join(self.workdir, chroma_persist_dir)
        self.client = chromadb.PersistentClient(path=chroma_persist_path)
        self.collection = self.client.get_collection(
            name="law_articles",
            embedding_function=ZhipuEmbeddingFunction()
        )
    def get_all(self):
        alldocs = self.collection.get()

        return alldocs['ids'],alldocs["documents"],alldocs["metadatas"]
    def keywords_update(self):
        ids,docs,metadatas = self.get_all()
        non_keywords = []
        zhipu_api_key = os.getenv("ZHIPU_API_KEY")
        client = ZhipuAiClient(api_key=zhipu_api_key)
        with open('prompts_keywords.txt', 'r', encoding='utf-8') as file:
            prompt = file.read()
            count_f = 0
        for i in range(len(ids)):
            id = ids[i]
            doc = docs[i]
            metadata=metadatas[i]
            metadata['keywords'] = str(self._generate_keywords(client, prompt, doc))
            if  metadata['keywords'] == '':
                non_keywords.append({
                    'id':id,
                    'doc':doc
                })
                count_f+=1
            
            self.collection.update(
            ids = [id],
            metadatas=[metadata]
            )
            print(f"已更新{i+1}条数据，其中{count_f}条不成功")
        return non_keywords
        
    def _generate_keywords( self,client: ZhipuAiClient, prompt,law_text: str) -> List[str]:
        """使用Zhipu GLM-4为法条文本生成关键词"""
        try:
            prompt = prompt
            response = client.chat.completions.create(
                model="glm-4-flashx",  # 使用GLM-4 (GLM-4.6在API中通常用glm-4标识)
                messages=[
                    {"role": "user", "content": f"法条内容：'{law_text}'\n\n{prompt}"}
                ],
                #max_tokens=60,  # 限制输出长度，关键词一般不长
                temperature=0.1, # 低温获取更稳定、相关的词
                # thinking={
                #          "type": "enabled",    # 启用深度思考模式
                # }
            )
            
            content = response.choices[0].message.content.strip()
            print(f"content:{content}")
            # 解析关键词，模型可能返回 "词1、词2、词3" 或 "词1, 词2, 词3" 或 "1. 词1 2. 词2"
            # 使用正则表达式提取所有中英文词汇
            keywords = re.findall(r'[\u4e00-\u9fa5a-zA-Z]+', content)
            print(f"keywords:{keywords}")
            # 返回前4个，符合prompt要求
            return keywords[:4] if keywords else []
        except Exception as e:
            print(f"Warning: Keyword generation failed for doc '{law_text[:50]}...': {e}")
            return [] # 发生错误时返回空列
        


In [30]:
up =  updater()

In [31]:
up.keywords_update()

content:家庭暴力，侵权案件，权益保护，家庭关系
keywords:['家庭暴力', '侵权案件', '权益保护', '家庭关系']
已更新1条数据，其中0条不成功
content:家庭暴力，侵权行为，人身伤害，精神损害
keywords:['家庭暴力', '侵权行为', '人身伤害', '精神损害']
已更新2条数据，其中0条不成功
content:家庭暴力，婚姻纠纷，侵权责任，社会管理
keywords:['家庭暴力', '婚姻纠纷', '侵权责任', '社会管理']
已更新3条数据，其中0条不成功
content:家庭暴力，司法干预，社会团体，经费保障
keywords:['家庭暴力', '司法干预', '社会团体', '经费保障']
已更新4条数据，其中0条不成功
content:家庭暴力，受害者保护，特殊群体，隐私保护
keywords:['家庭暴力', '受害者保护', '特殊群体', '隐私保护']
已更新5条数据，其中0条不成功
content:家庭暴力，宣传教育，学校教育，社会宣传
keywords:['家庭暴力', '宣传教育', '学校教育', '社会宣传']
已更新6条数据，其中0条不成功
content:家庭暴力，司法培训，医疗记录，受害者保护
keywords:['家庭暴力', '司法培训', '医疗记录', '受害者保护']
已更新7条数据，其中0条不成功
content:家庭暴力，预防工作，社区协助，政府责任
keywords:['家庭暴力', '预防工作', '社区协助', '政府责任']
已更新8条数据，其中0条不成功
content:家庭暴力，社会服务，心理健康，预防教育
keywords:['家庭暴力', '社会服务', '心理健康', '预防教育']
已更新9条数据，其中0条不成功
content:家庭纠纷，调解，预防，暴力
keywords:['家庭纠纷', '调解', '预防', '暴力']
已更新10条数据，其中0条不成功
content:用人单位，家庭暴力，调解，教育
keywords:['用人单位', '家庭暴力', '调解', '教育']
已更新11条数据，其中0条不成功
content:家庭教育，监护责任，未成年人，家庭暴力
keywords:['家庭教育', '监护责任', '未成年人', '家庭暴力'

[]

In [110]:
a[0]

'中华人民共和国反家庭暴力法_1'

In [112]:
str(["d","d","v"])

"['d', 'd', 'v']"

In [109]:
b[0]

'中华人民共和国反家庭暴力法第1条：为了预防和制止家庭暴力，保护家庭成员的合法权益，维护平等、和睦、文明的家庭关系，促进家庭和谐、社会稳定，制定本法。'

In [113]:
c[0]['keywords']="wdwc"
c[0]['keywords']

'wdwc'

In [20]:
r = retriver.collection.get(where ={
    "law_name":"中华人民共和国反家庭暴力法"
})

In [22]:
r['documents']

['中华人民共和国反家庭暴力法第1条：为了预防和制止家庭暴力，保护家庭成员的合法权益，维护平等、和睦、文明的家庭关系，促进家庭和谐、社会稳定，制定本法。',
 '中华人民共和国反家庭暴力法第2条：本法所称家庭暴力，是指家庭成员之间以殴打、捆绑、残害、限制人身自由以及经常性谩骂、恐吓等方式实施的身体、精神等侵害行为。',
 '中华人民共和国反家庭暴力法第3条：家庭成员之间应当互相帮助，互相关爱，和睦相处，履行家庭义务。反家庭暴力是国家、社会和每个家庭的共同责任。国家禁止任何形式的家庭暴力。',
 '中华人民共和国反家庭暴力法第4条：县级以上人民政府负责妇女儿童工作的机构，负责组织、协调、指导、督促有关部门做好反家庭暴力工作。县级以上人民政府有关部门、司法机关、人民团体、社会组织、居民委员会、村民委员会、企业事业单位，应当依照本法和有关法律规定，做好反家庭暴力工作。各级人民政府应当对反家庭暴力工作给予必要的经费保障。',
 '中华人民共和国反家庭暴力法第5条：反家庭暴力工作遵循预防为主，教育、矫治与惩处相结合原则。反家庭暴力工作应当尊重受害人真实意愿，保护当事人隐私。未成年人、老年人、残疾人、孕期和哺乳期的妇女、重病患者遭受家庭暴力的，应当给予特殊保护。',
 '中华人民共和国反家庭暴力法第6条：国家开展家庭美德宣传教育，普及反家庭暴力知识，增强公民反家庭暴力意识。工会、共产主义青年团、妇女联合会、残疾人联合会应当在各自工作范围内，组织开展家庭美德和反家庭暴力宣传教育。广播、电视、报刊、网络等应当开展家庭美德和反家庭暴力宣传。学校、幼儿园应当开展家庭美德和反家庭暴力教育。',
 '中华人民共和国反家庭暴力法第7条：县级以上人民政府有关部门、司法机关、妇女联合会应当将预防和制止家庭暴力纳入业务培训和统计工作。医疗机构应当做好家庭暴力受害人的诊疗记录。',
 '中华人民共和国反家庭暴力法第8条：乡镇人民政府、街道办事处应当组织开展家庭暴力预防工作，居民委员会、村民委员会、社会工作服务机构应当予以配合协助。',
 '中华人民共和国反家庭暴力法第9条：各级人民政府应当支持社会工作服务机构等社会组织开展心理健康咨询、家庭关系指导、家庭暴力预防知识教育等服务。',
 '中华人民共和国反家庭暴力法第10条：人民调解组织应当依法调解家庭纠纷，预防和减少家庭暴力的发生。',
 '中华人民

In [90]:
def _generate_keywords( client: ZhipuAiClient, prompt,law_text: str) -> List[str]:
        """使用Zhipu GLM-4为法条文本生成关键词"""
        try:
            prompt = prompt
            response = client.chat.completions.create(
                model="glm-4-flashx",  # 使用GLM-4 (GLM-4.6在API中通常用glm-4标识)
                messages=[
                    {"role": "user", "content": f"法条内容：'{law_text}'\n\n{prompt}"}
                ],
                #max_tokens=60,  # 限制输出长度，关键词一般不长
                temperature=0.1, # 低温获取更稳定、相关的词
                # thinking={
                #          "type": "enabled",    # 启用深度思考模式
                # }
            )
            
            content = response.choices[0].message.content.strip()
            print(f"content:{content}")
            # 解析关键词，模型可能返回 "词1、词2、词3" 或 "词1, 词2, 词3" 或 "1. 词1 2. 词2"
            # 使用正则表达式提取所有中英文词汇
            keywords = re.findall(r'[\u4e00-\u9fa5a-zA-Z]+', content)
            print(f"keywords:{keywords}")
            # 返回前4个，符合prompt要求
            return keywords[:4] if keywords else []
        except Exception as e:
            print(f"Warning: Keyword generation failed for doc '{law_text[:50]}...': {e}")
            return [] # 发生错误时返回空列

In [91]:
import re
zhipu_api_key = os.getenv("ZHIPU_API_KEY")
client = ZhipuAiClient(api_key=zhipu_api_key)
# 读取prompt
with open('prompts_keywords.txt', 'r', encoding='utf-8') as file:
    prompt = file.read()
_generate_keywords(client, prompt, r['documents'][0])

content:家庭暴力，合法权益，家庭关系，社会稳定
keywords:['家庭暴力', '合法权益', '家庭关系', '社会稳定']


['家庭暴力', '合法权益', '家庭关系', '社会稳定']

In [38]:
r['documents'][0]

'中华人民共和国反家庭暴力法第1条：为了预防和制止家庭暴力，保护家庭成员的合法权益，维护平等、和睦、文明的家庭关系，促进家庭和谐、社会稳定，制定本法。'

In [37]:
prompt

'你是一名法律助理。\n我将给你一条法条或法规内容。\n请输出4个简短词语，描述该条文的适用场景、案件类型或使用情境。\n这些词语应简短，不解释法律条文。\n示例：交通事故、合同纠纷、环境污染、劳动解雇等。\n输出格式为：“词1，词2，词3，词4”'

In [5]:
import tiktoken

def count_tokens(text, model="gpt-3.5-turbo"):
    # 自动获取模型对应的编码方式
    encoding = tiktoken.encoding_for_model(model)
    # 编码文本并统计 Token 数量
    token_count = len(encoding.encode(text))
    return token_count

# 示例：统计一段文本的 Token 数
token_num = count_tokens(txt)
print(f"Token 数量：{token_num}")

Token 数量：3192


In [96]:
import json

def generate_jsonl(user_contents, output_file):
    """
    生成符合要求的JSONL文件
    
    参数:
        user_contents (list): 用户内容列表，每个元素为user角色的content值
        output_file (str): 输出的JSONL文件路径
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        for i, content in enumerate(user_contents, start=1):
            # 构建单条JSON数据
            json_data = {
                "custom_id": f"request-{i}",
                "method": "POST",
                "url": "/v4/chat/completions",
                "body": {
                    "model": "glm-4-flashx-250414",
                    "messages": [
                        {
                            "role": "system",
                            "content": "你是一名法律助理。"
                        },
                        {
                            "role": "user",
                            "content":  f"{prompt}\n\n法条内容：'{content}'\n"
                        }
                    ]
                }
            }
            # 写入JSONL文件（每条JSON单独一行）
            f.write(json.dumps(json_data, ensure_ascii=False) + '\n')

In [97]:
b[0]

'中华人民共和国反家庭暴力法第1条：为了预防和制止家庭暴力，保护家庭成员的合法权益，维护平等、和睦、文明的家庭关系，促进家庭和谐、社会稳定，制定本法。'

In [98]:
# 准备用户内容列表
user_contents = b

# 生成JSONL文件
generate_jsonl(user_contents, "batch_process.jsonl")

In [99]:
from zai import ZhipuAiClient

client = ZhipuAiClient(api_key=zhipu_api_key)

# 上传批处理文件
file_object = client.files.create(
    file=open("batch_process.jsonl", "rb"),
    purpose="batch"
)
print(file_object)

FileObject(id='1762581729_d110e7f4411542d5a17446a36782ef21', bytes=13373702, created_at=1762581729, filename='batch_process.jsonl', object='file', purpose='batch', status=None, status_details=None)


In [100]:
# 创建批处理任务
batch = client.batches.create(
    input_file_id=file_object.id,
    endpoint="/v4/chat/completions",
    auto_delete_input_file=True,
    metadata={
        "description": "为每一条法规添加场景关键词",
        "project": "law_keywords"
    }
)
print(batch)

Batch(id='batch_1987038202176479232', completion_window=None, created_at=1762581799440, endpoint='/v4/chat/completions', input_file_id='1762581729_d110e7f4411542d5a17446a36782ef21', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=None, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': '为每一条法规添加场景关键词', 'project': 'law_keywords'}, output_file_id=None, request_counts=BatchRequestCounts(completed=None, failed=None, total=13722))
